# Pre-processing California Allocation data for WaDE upload.
Date Updated: 02/14/2023
Purpose:  To pre-process the California data into one master file for simple DataFrame creation and extraction

Notes:
- asdf

In [1]:
#Needed Libararies

# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# visulizaiton
import matplotlib.pyplot as plt
import seaborn as sns

# API retrieval
import requests
import json

# Cleanup
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x) # suppress scientific notation in Pandas

In [2]:
# Working Directory
workingDir = "G:/Shared drives/WaDE Data/California/WaterAllocation/RawInputData"
os.chdir(workingDir)

## Import Input Data

In [3]:
# POD Data
fileInput = "water_rights_list_2023-02-13_input.zip"
dfinPOD = pd.read_csv(fileInput, compression='zip')

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfinPOD:
    dfinPOD['WaDEUUID'] = "caD" + dfinPOD.index.astype(str)
    dfinPOD.to_csv('water_rights_list_2023-02-13_input.zip', compression='zip', index=False)

print(len(dfinPOD))
dfinPOD.head(1)

C:\Users\rjame\AppData\Local\Temp\ipykernel_6680\3395579190.py:3: DtypeWarning: Columns (2,3,4,7,9,14,20,22,35,36,38,41,43,45,46,47,49,54,63,68,74,100,104,135,136,137,138,139,142,143,144,145,146,147,148,149,150,151,152,153,155,156,157,158) have mixed types. Specify dtype option on import or set low_memory=False.
  dfinPOD = pd.read_csv(fileInput, compression='zip')


65243


,wr_water_right_id,application_number,certificate_id,permit_id,license_id,water_right_type,water_right_status,priority_date,receipt_date,rejection_date,application_recd_date,application_acceptance_date,project_type,record_summary,incomplete_statement,number_of_protests,agent_name,agent_entity_type,application_primary_owner,primary_owner_entity_type,sub_type,ini_reported_div_amount,ini_reported_div_unit,face_value_amount,face_value_units,fee_received,appl_fee_amount,appl_fee_amt_recd,max_dd_appl,max_dd_units,max_dd_ann,max_storage,max_taken_from_source,year_diversion_commenced,max_beneficially_used,supplemental_statement_cycle,type_of_diversion_facility,quantity_of_water_diverted,qow_diverted_unit,quantity_measurement_year,max_rate_of_diversion,max_rate_of_div_unit,recent_water_use_min,water_use_min_unit,recent_water_use_max,water_use_max_unit,request_for_revocation_recd,state_well_number,drilled_well_year,surface_water_diversions,depth_of_well,relationship_type,party_id,effective_from_date,effective_to_date,primary_owner_name,official_mail_receiver,count_npo_or_other,effective_date,update_datetime,use_code,use_status,number_of_residences,seperately_owned,use_population,use_population_people,estimated_use_per_person,use_population_stock,type_of_stock,area_for_inci_irrigation,use_net_acreage,use_gross_acreage,use_direct_div_annual_amount,use_direct_diversion_rate,use_direct_div_rate_units,pou_development_status,direct_div_season_start,direct_div_season_end,use_storage_amount,storage_season_start,storage_season_end,season_direct_div_rate,season_storage_amount,season_direct_div_aa,direct_div_season_status,collection_season_status,use_count,pod_number,pod_id,pod_status,source_type,pod_name,pod_type,diversion_works_status,stream_classification,direct_div_amount,direct_diversion_rate,direct_div_rate_unit,storage_amount,diversion_rate_to_off_stream,off_sto_div_rate_unit,pod_last_update_date,pod_count,objectid,pod_number_gis,has_opod,appl_pod,pod_id_gis,county,parcel_number,sp_zone,diversion_site_name,north_coord,east_coord,latitude,longitude,quarter_quarter,quarter,section_classifier,section_number,township_number,township_direction,range_number,range_direction,meridian,location_method,special_use_area,source_name,trib_desc,watershed,huc_12_number,huc_12_name,huc_8_number,huc_8_name,quad_map_name,quad_map_number,permit_original_issue_date,complete_construction_date,complete_applic_water_date,license_original_issue_date,water_right_description,program_unit,license_request_type,license_requested_date,inspection_date,report_date,offer_sent_date,accepted_offer_date,petition_id,petition_type,petition_status_type,date_received,date_completed,pet_last_update_date,num_of_petitions,enf_case_number,enf_case_start_date,enf_case_closure_date,enf_case_county,number_of_enforcement_case,WaDEUUID
0,0,NaN,NaN,NaN,NaN,Not Determined,Closed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,12/31/2021,31-DEC-21 08.25.23.893238 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ENF04329,8/26/2022,2/7/2023,NaN,102,caD0


In [4]:
# Clean data a little
dfinPOD = dfinPOD.fillna("") # remove nan values
dfinPOD['use_direct_diversion_rate'] = pd.to_numeric(dfinPOD['use_direct_diversion_rate'], errors='coerce').fillna(0) # make sure this is numeric.
print(len(dfinPOD))
dfinPOD.head(1)

65243


,wr_water_right_id,application_number,certificate_id,permit_id,license_id,water_right_type,water_right_status,priority_date,receipt_date,rejection_date,application_recd_date,application_acceptance_date,project_type,record_summary,incomplete_statement,number_of_protests,agent_name,agent_entity_type,application_primary_owner,primary_owner_entity_type,sub_type,ini_reported_div_amount,ini_reported_div_unit,face_value_amount,face_value_units,fee_received,appl_fee_amount,appl_fee_amt_recd,max_dd_appl,max_dd_units,max_dd_ann,max_storage,max_taken_from_source,year_diversion_commenced,max_beneficially_used,supplemental_statement_cycle,type_of_diversion_facility,quantity_of_water_diverted,qow_diverted_unit,quantity_measurement_year,max_rate_of_diversion,max_rate_of_div_unit,recent_water_use_min,water_use_min_unit,recent_water_use_max,water_use_max_unit,request_for_revocation_recd,state_well_number,drilled_well_year,surface_water_diversions,depth_of_well,relationship_type,party_id,effective_from_date,effective_to_date,primary_owner_name,official_mail_receiver,count_npo_or_other,effective_date,update_datetime,use_code,use_status,number_of_residences,seperately_owned,use_population,use_population_people,estimated_use_per_person,use_population_stock,type_of_stock,area_for_inci_irrigation,use_net_acreage,use_gross_acreage,use_direct_div_annual_amount,use_direct_diversion_rate,use_direct_div_rate_units,pou_development_status,direct_div_season_start,direct_div_season_end,use_storage_amount,storage_season_start,storage_season_end,season_direct_div_rate,season_storage_amount,season_direct_div_aa,direct_div_season_status,collection_season_status,use_count,pod_number,pod_id,pod_status,source_type,pod_name,pod_type,diversion_works_status,stream_classification,direct_div_amount,direct_diversion_rate,direct_div_rate_unit,storage_amount,diversion_rate_to_off_stream,off_sto_div_rate_unit,pod_last_update_date,pod_count,objectid,pod_number_gis,has_opod,appl_pod,pod_id_gis,county,parcel_number,sp_zone,diversion_site_name,north_coord,east_coord,latitude,longitude,quarter_quarter,quarter,section_classifier,section_number,township_number,township_direction,range_number,range_direction,meridian,location_method,special_use_area,source_name,trib_desc,watershed,huc_12_number,huc_12_name,huc_8_number,huc_8_name,quad_map_name,quad_map_number,permit_original_issue_date,complete_construction_date,complete_applic_water_date,license_original_issue_date,water_right_description,program_unit,license_request_type,license_requested_date,inspection_date,report_date,offer_sent_date,accepted_offer_date,petition_id,petition_type,petition_status_type,date_received,date_completed,pet_last_update_date,num_of_petitions,enf_case_number,enf_case_start_date,enf_case_closure_date,enf_case_county,number_of_enforcement_case,WaDEUUID
0,0,,,,,Not Determined,Closed,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0,12/31/2021,31-DEC-21 08.25.23.893238 AM,,,,,,,,,,,,,,0.00000,,,,,,,,,,,,,0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,ENF04329,8/26/2022,2/7/2023,,102,caD0


In [5]:
# Create VariableSpecificCv value
def createVariableSpecificCv(unit):
    outString = ""
    if unit == "Cubic Feet per Second":
        outString = "CAwr_V1"
    if unit == "Gallons per Day":
        outString = "CAwr_V1"
    if unit == "Acre-feet per Year":
        outString = "CAwr_V1"
    if unit == "Gallons per Minute":
        outString = "CAwr_V1"
    if unit == 'Acre-feet':
        outString = "CAwr_V2"
    if unit == 'Gallons':
        outString = "CAwr_V2"
    else:
        outString = "CAwr_V1"

    return(outString)

dfinPOD['in_VariableSpecificUUID'] = dfinPOD.apply(lambda row: createVariableSpecificCv(row['use_direct_div_rate_units']), axis=1)
dfinPOD['in_VariableSpecificUUID'].unique()

array(['CAwr_V1', 'CAwr_V2'], dtype=object)

In [6]:
# convert all flow values to CFS
def convertFlowFunc(val, unit):
    CFS_Value = None
    if unit == "Cubic Feet per Second":
        CFS_Value = val
    if unit == "Gallons per Day":
        CFS_Value = val / (646316.883)
    if unit == "Acre-feet per Year":
        CFS_Value = val / (723.968)
    if unit == "Gallons per Minute":
        CFS_Value = val / (448.83117)
    return(CFS_Value)

dfinPOD['CFS_Value'] = dfinPOD.apply(lambda row: convertFlowFunc(row['use_direct_diversion_rate'], row['use_direct_div_rate_units']), axis=1)
dfinPOD['CFS_Value'].unique()

array([           nan, 1.50000000e-01, 2.50000000e+00, ...,
       2.11660879e+01, 1.17248987e-02, 7.53000000e+00])

In [7]:
# convert all volume values to AF
def convertVolumeFunc(val, unit):
    AF_Value = None
    if unit == 'Acre-feet':
        AF_Value = val
    if unit == 'Gallons':
        AF_Value = val / (325850.943)
    return(AF_Value)

dfinPOD['AF_Value'] = dfinPOD.apply(lambda row: convertVolumeFunc(row['use_direct_diversion_rate'], row['use_direct_div_rate_units']), axis=1)
dfinPOD['AF_Value'].unique()

array([      nan, 0.000e+00, 1.300e-01, 3.650e+02, 2.062e+01])

## POD Sites Data

In [8]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfinPOD['WaDEUUID']

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = dfinPOD['source_name']
df['in_WaterSourceNativeID'] = "" # create customID for temp solution
df['in_WaterSourceTypeCV'] = dfinPOD['source_type']

# Site Info
df['in_RegulatoryOverlayUUIDs'] = ""
df['in_WaterSourceUUID'] = "" # ???
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = dfinPOD['location_method']
df['in_County'] = dfinPOD['county']
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = dfinPOD['huc_12_number']
df['in_HUC8'] = dfinPOD['huc_8_number']
df['in_Latitude'] = dfinPOD['latitude']
df['in_Longitude'] = dfinPOD['longitude']
df['in_NHDNetworkStatusCV'] = dfinPOD['longitude']
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POD"
df['in_SiteName'] = dfinPOD['pod_name']
df['in_SiteNativeID'] = dfinPOD['pod_id'].fillna(0).replace("",0)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = "WaDE_Unspecified"
df['in_StateCV'] = "CA"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_MethodUUID'] = "CAwr_M1"
df['in_OrganizationUUID'] = "CAwr_O1"
df['in_SiteUUID'] = "" # ???
df['in_VariableSpecificUUID'] =  dfinPOD['in_VariableSpecificUUID']
df['in_AllocationApplicationDate'] = dfinPOD['application_recd_date']
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDateID'] = ""
df['in_AllocationFlow_CFS'] = dfinPOD['CFS_Value'].fillna(0).astype(float) # see above for conversion
df['in_AllocationLegalStatusCV'] = dfinPOD['water_right_status']
df['in_AllocationNativeID'] =  dfinPOD['application_number']
df['in_AllocationOwner'] = dfinPOD['primary_owner_name']
df['in_AllocationPriorityDate'] = dfinPOD['priority_date']
df['in_AllocationTimeframeEnd'] = dfinPOD['direct_div_season_end']
df['in_AllocationTimeframeStart'] = dfinPOD['direct_div_season_start']
df['in_AllocationTypeCV'] = dfinPOD['water_right_type'].astype(str) +"_" + dfinPOD['sub_type'].astype(str)
df['in_AllocationVolume_AF'] = dfinPOD['AF_Value'].fillna(0).astype(float) # see above for conversion
df['in_BeneficialUseCategory'] = dfinPOD['use_code']
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = dfinPOD['use_net_acreage']
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = dfinPOD['use_population']
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = "https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/EWServlet?OWASP_CSRFTOKEN=2F29-W6H3-FM0Y-I9W5-P5G3-W5MG-VN8J-1DL2&Page_From=EWWaterRightPublicSearch.jsp&Redirect_Page=EWWaterRightPublicSearchResults.jsp&Object_Expected=EwrimsSearchResult&Object_Created=EwrimsSearch&Object_Criteria=&Purpose=&subTypeCourtAdjSpec=&subTypeOtherSpec=&appNumber=" + dfinPOD['application_number'].astype(str)

print(len(df))
df.head()

65243


,WaDEUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_RegulatoryOverlayUUIDs,in_WaterSourceUUID,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_MethodUUID,in_OrganizationUUID,in_SiteUUID,in_VariableSpecificUUID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDateID,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,caD0,,,,,,,,,,,,4326,,,,,,,,POD,,0.00000,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Closed,,,,,,Not Determined_,0.00000,,,,,,,0,,,,,,,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
1,caD1,,,,LAKE DOMINGO,,Surface,,,,GIS_NE,San Diego,4326,,181002000000.00000,18100202.00000,32.61440,-116.27260,-116.27260,,POD,LAKE DOMINGO,60498.00000,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,4/18/2013,,,,,,,,0.15000,Cancelled,T032025,"569 EAST COUNTY BOULEVARD, LLC",,31-Dec,1-Jan,Temporary Permit_,0.00000,Dust Control,,,,,,0,,0.00000,,,,0.00000,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
2,caD2,,,,ICY SPRING NO 2,,,,,,DD_NE,Los Angeles,4326,,180902000000.00000,18090206.00000,34.57797,-118.28921,-118.28921,,POD,,34881.00000,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Licensed,A000016,LILAC HILLS ESTATES LP,,31-Dec,1-Jan,Appropriative_,0.00000,Domestic,,,,,,0,,0.00000,,,,0.00000,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
3,caD3,,,,SACRAMENTO RIVER,,,,,,DD_NE,Glenn,4326,,180202000000.00000,18020157.00000,39.79451,-122.05447,-122.05447,,POD,,28036.00000,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Licensed,A000018,GLENN-COLUSA IRRIGATION DISTRICT,,1-Nov,1-Mar,Appropriative_,0.00000,Irrigation,,,,,,0,,6047.70000,,,,,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
4,caD4,,,,SAN JOAQUIN RIVER,,,,,,DD_NE,Madera,4326,,180400000000.00000,18040001.00000,37.00260,-119.70722,-119.70722,,POD,,23233.00000,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Licensed,A000023,US BUREAU OF RECLAMATION,,1-Jul,1-Apr,Appropriative_,0.00000,Domestic,,,,,,0,,0.00000,,,,0.00000,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...


In [9]:
# Concatenate dataframes
frames = [df] # add dataframes here
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(outdf))

65243


## Clean output dataframes

In [10]:
outdf.head()

,WaDEUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_RegulatoryOverlayUUIDs,in_WaterSourceUUID,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_MethodUUID,in_OrganizationUUID,in_SiteUUID,in_VariableSpecificUUID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDateID,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,caD0,,,,,,,,,,,,4326,,,,,,,,POD,,0.00000,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Closed,,,,,,Not Determined_,0.00000,,,,,,,0,,,,,,,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
1,caD1,,,,LAKE DOMINGO,,Surface,,,,GIS_NE,San Diego,4326,,181002000000.00000,18100202.00000,32.61440,-116.27260,-116.27260,,POD,LAKE DOMINGO,60498.00000,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,4/18/2013,,,,,,,,0.15000,Cancelled,T032025,"569 EAST COUNTY BOULEVARD, LLC",,31-Dec,1-Jan,Temporary Permit_,0.00000,Dust Control,,,,,,0,,0.00000,,,,0.00000,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
2,caD2,,,,ICY SPRING NO 2,,,,,,DD_NE,Los Angeles,4326,,180902000000.00000,18090206.00000,34.57797,-118.28921,-118.28921,,POD,,34881.00000,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Licensed,A000016,LILAC HILLS ESTATES LP,,31-Dec,1-Jan,Appropriative_,0.00000,Domestic,,,,,,0,,0.00000,,,,0.00000,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
3,caD3,,,,SACRAMENTO RIVER,,,,,,DD_NE,Glenn,4326,,180202000000.00000,18020157.00000,39.79451,-122.05447,-122.05447,,POD,,28036.00000,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Licensed,A000018,GLENN-COLUSA IRRIGATION DISTRICT,,1-Nov,1-Mar,Appropriative_,0.00000,Irrigation,,,,,,0,,6047.70000,,,,,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
4,caD4,,,,SAN JOAQUIN RIVER,,,,,,DD_NE,Madera,4326,,180400000000.00000,18040001.00000,37.00260,-119.70722,-119.70722,,POD,,23233.00000,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Licensed,A000023,US BUREAU OF RECLAMATION,,1-Jul,1-Apr,Appropriative_,0.00000,Domestic,,,,,,0,,0.00000,,,,0.00000,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...


In [11]:
# Clean Up AllocationTypeCV
def assignAllocationTypeCV(val):
    val = str(val).strip()
    val = val.rstrip('_') # remove trailing "_"
    val = val.rstrip(',') # remove trailing "_" # remove trailing commas
    if val == "" or pd.isnull(val):
        outString = "WaDE_Unspecified"
    else:
        outString = val
    return outString

outdf['in_AllocationTypeCV'] = outdf.apply(lambda row: assignAllocationTypeCV(row['in_AllocationTypeCV']), axis=1)
outdf['in_AllocationTypeCV'].unique()

array(['Not Determined', 'Temporary Permit', 'Appropriative',
       'Appropriative (State Filing)', 'Stockpond',
       'Registration Domestic', 'Cert of Right - Power', 'Federal Claims',
       'Federal Stockponds', 'Groundwater Recordation',
       'Registration Livestock', 'Statement of Div and Use',
       'Statement of Div and Use_RIPERIAN',
       'Statement of Div and Use_RIPERIAN,PRE1914',
       'Statement of Div and Use_PRE1914',
       'Statement of Div and Use_OTHER',
       'Statement of Div and Use_PRE1914,OTHER',
       'Statement of Div and Use_COURTADJ',
       'Statement of Div and Use_RIPERIAN,PRE1914,OTHER',
       'Statement of Div and Use_RIPERIAN,PRE1914,COURTADJ,OTHER',
       'Statement of Div and Use_RIPERIAN,PRE1914,COURTADJ',
       'Statement of Div and Use_PRE1914,COURTADJ',
       'Statement of Div and Use_RIPERIAN,OTHER',
       'Statement of Div and Use_RIPERIAN,COURTADJ', 'Section 12 File',
       'Waste Water Change', 'Adjudicated', 'Non Jurisdiction

In [12]:
# Clean Up iAllocationLegalStatusCV
def assignAllocationLegalStatusCV(val):
    val = str(val).strip()
    if val == "" or pd.isnull(val):
        outString = "WaDE_Unspecified"
    else:
        outString = val
    return outString

outdf['in_AllocationLegalStatusCV'] = outdf.apply(lambda row: assignAllocationLegalStatusCV(row['in_AllocationLegalStatusCV']), axis=1)
outdf['in_AllocationLegalStatusCV'].unique()

array(['Closed', 'Cancelled', 'Licensed', 'Revoked', 'Permitted',
       'Pending', 'Certified', 'Registered', 'Rejected', 'Claimed',
       'Inactive', 'Claimed - Local Oversight', 'Completed', 'Active',
       'Adjudicated', 'WaDE_Unspecified', 'Withdrawn'], dtype=object)

In [13]:
# ExemptOfVolumeFlowPriority
# check for keywrods 'RIPERIAN', 'PRE1914', or 'Statement of Div and Use' in AllocationTypeCV
def assignExemptOfVolumeFlowPriority(valString):
    valString = str(valString).strip()
    testList = ['RIPERIAN', 'PRE1914', 'Statement of Div and Use']
    checkStr = any(ele in valString for ele in testList)
    if checkStr == True:
        outString = "1"
    else:
        outString = "0"
    return outString

outdf['in_ExemptOfVolumeFlowPriority'] = outdf.apply(lambda row: assignExemptOfVolumeFlowPriority(row['in_AllocationTypeCV']), axis=1)
outdf['in_ExemptOfVolumeFlowPriority'].unique()

array(['0', '1'], dtype=object)

In [14]:
# For filling in Unspecified when null

def assignBeneficialUseCategory(val):
    val = str(val).strip()
    if val == "" or pd.isnull(val):
        outString = "WaDE_Unspecified"
    else:
        outString = val
    return outString

outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: assignBeneficialUseCategory(row['in_BeneficialUseCategory']), axis=1)
outdf['in_BeneficialUseCategory'].unique()

array(['WaDE_Unspecified', 'Dust Control', 'Domestic', 'Irrigation',
       'Power', 'Municipal',
       'Fish and Wildlife Preservation and Enhancement', 'Stockwatering',
       'Industrial', 'Mining', 'Recreational', 'Fire Protection',
       'Incidental Power', 'Frost Protection', 'Aquaculture',
       'Snow Making', 'Milling', 'Heat Control', 'Other', 'Aesthetic',
       'Water Quality'], dtype=object)

In [15]:
# Update datatype of Priority Date to fit WaDE 2.0 structure
def formatDateString(inString1):
    inString = str(inString1).strip()
    try:
        if inString == "" or pd.isnull(inString):
            valndf = ""
        else:
            valD = pd.to_datetime(inString)
            valnDd = valD.date()
            valndf = valnDd.strftime('%m/%d/%Y')
    except:
        valndf = ""
    return valndf

outdf['in_AllocationPriorityDate'] = outdf.apply(lambda row: formatDateString(row['in_AllocationPriorityDate']), axis=1)
outdf['in_AllocationPriorityDate'].unique()

array(['', '07/22/1915', '04/17/1916', ..., '04/22/2022', '02/08/2023',
       '02/10/2023'], dtype=object)

In [16]:
# Convert in_Latitude & in_Longitude to numeric
outdf['in_Latitude'] = pd.to_numeric(outdf['in_Latitude'], errors='coerce').fillna(0).astype(float)
print(outdf['in_Latitude'].unique())
outdf['in_Longitude'] = pd.to_numeric(outdf['in_Longitude'], errors='coerce').fillna(0).astype(float)
print(outdf['in_Longitude'].unique())

[ 0.         32.6144     34.57797187 ... 40.568306   39.73614055
 39.22815332]
[   0.        -116.2726    -118.2892096 ... -123.055788  -123.5046908
 -121.8252882]


In [17]:
# Convert in_PopulationServed to numeric
outdf['in_PopulationServed'] = pd.to_numeric(outdf['in_PopulationServed'], errors='coerce').fillna(0).astype(int)
outdf['in_PopulationServed'].unique()

array([      0,    6000,    7030,    3294,  153600,   42912,    2500,
        223296,    7727,       1,    3681,     813,    1000,      35,
         41591,   57272,  202000,  112000,  315000,   56500,   69000,
        248000,   91923,    1585,   35795,    1840,     250,  106926,
         15000,  106700,   78409,   16874,   49597,  106949,   12927,
         53409,  219068,   17170,    8693,    2722,    6960,    4914,
          3011,   22543,   13320,       6,    2016,   40883,   16852,
           800,   12998,   50750,    9733,    2281,   30450,   13000,
          5800,    5731,   13849,   86548,    1750,  755000,  708590,
        578000,    6800,    1634,   82389,    3382,   87000,   91829,
         11591,    3256,    5421,       3,      17,     347,      51,
        127841,  109375,  186379,   89704,      46,   18852,   41960,
         27233,       2,  152000,       8,      10,  933710,  130400,
        348800,   21591,  102000,  620745,   38000,  613800,      14,
        329273,   95

In [18]:
# Convert in_IrrigatedAcreage to numeric
outdf['in_IrrigatedAcreage'] = pd.to_numeric(outdf['in_IrrigatedAcreage'], errors='coerce').fillna(0).astype(float)
outdf['in_IrrigatedAcreage'].unique()

array([   0. , 6047.7, 7000. , ...,   11.9,   47.2,   20.5])

In [19]:
# For creating CoordinateMethodCV
def assignCoordinateMethodCV(colrowValue):
    colrowValue = str(colrowValue).strip()
    if colrowValue == "" or colrowValue == "nan" or pd.isnull(colrowValue):
        outList = "WaDE_Unspecified"
    else:
        try:
            outList = str(colrowValue).strip()
        except:
            outList = "WaDE_Unspecified"
    return outList

outdf['in_CoordinateMethodCV'] = df.apply(lambda row: assignCoordinateMethodCV(row['in_CoordinateMethodCV']), axis=1)
outdf['in_CoordinateMethodCV'].unique()

array(['WaDE_Unspecified', 'GIS_NE', 'DD_NE', 'GIS_LL', 'GIS_CLICK',
       'GIS_DDO', 'GIS_BDO', 'DD_QQS', 'DD_S', 'DD_QS', 'GIS_UTM'],
      dtype=object)

In [20]:
# For creating SiteName
def assignSiteName(colrowValue):
    colrowValue = str(colrowValue).strip()
    if colrowValue == "" or colrowValue == "nan" or pd.isnull(colrowValue):
        outList = "WaDE_Unspecified"
    else:
        try:
            outList = str(colrowValue).strip()
        except:
            outList = "WaDE_Unspecified"
    return outList

outdf['in_SiteName'] = df.apply(lambda row: assignSiteName(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

array(['WaDE_Unspecified', 'LAKE DOMINGO', 'Ingalls Swamp', ..., '95',
       'Lake Cascade', 'Reservoir 24 Dam'], dtype=object)

In [21]:
# For creating SiteNativeID
def assignSiteNativeID(colrowValue):
    colrowValue = int(colrowValue)
    if colrowValue == "" or colrowValue == "nan" or colrowValue == 0 or pd.isnull(colrowValue):
        outString = "WaDE_Unspecified"
    else:
        outString = "CA" + str(colrowValue).strip()
    return outString

outdf['in_SiteNativeID'] = df.apply(lambda row: assignSiteNativeID(row['in_SiteNativeID']), axis=1)
outdf['in_SiteNativeID'].unique()

array(['WaDE_Unspecified', 'CA60498', 'CA34881', ..., 'CA84459',
       'CA84499', 'CA84502'], dtype=object)

In [22]:
# For creating WaterSourceName
def assignWaterSourceName(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outList = "WaDE_Unspecified"
    else:
        try:
            outList = str(colrowValue).strip()
        except:
            outList = "WaDE_Unspecified"
    return outList

outdf['in_WaterSourceName'] = df.apply(lambda row: assignWaterSourceName(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

array(['WaDE_Unspecified', 'LAKE DOMINGO', 'ICY SPRING NO 2', ...,
       'Unnamed stream, Tributary to Mad River, Thence Pacific Ocean',
       'Kellogg Creek', 'Offset wells on Russian River'], dtype=object)

In [23]:
# For creating WaterSourceTypeCV
def assignWaterSourceTypeCV(colrowValue):
    if colrowValue == "" or pd.isnull(colrowValue):
        outList = "WaDE_Unspecified"
    else:
        try:
            outList = str(colrowValue).strip()
        except:
            outList = "WaDE_Unspecified"
    return outList

outdf['in_WaterSourceTypeCV'] = df.apply(lambda row: assignWaterSourceTypeCV(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

array(['WaDE_Unspecified', 'Surface', 'Subsurface'], dtype=object)

In [24]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "WaDECA_WS" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        ml = dfWaterSourceNativeID.loc[(dfWaterSourceNativeID['in_WaterSourceName'] == A) & 
                                       (dfWaterSourceNativeID['in_WaterSourceTypeCV'] == B), 'in_WaterSourceNativeID']
        if not (ml.empty):  # check if the series is empty
            outList = ml.iloc[0]
        else:
            outList = ''
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

array(['WaDECA_WS1', 'WaDECA_WS2', 'WaDECA_WS3', ..., 'WaDECA_WS9654',
       'WaDECA_WS9655', 'WaDECA_WS9656'], dtype=object)

## Review and Export

In [25]:
outdf.dtypes

WaDEUUID                           object
in_Geometry                        object
in_GNISFeatureNameCV               object
in_WaterQualityIndicatorCV         object
in_WaterSourceName                 object
                                    ...  
in_PopulationServed                 int32
in_PowerType                       object
in_PrimaryBeneficialUseCategory    object
in_SDWISIdentifierCV               object
in_WaterAllocationNativeURL        object
Length: 65, dtype: object

In [26]:
outdf

,WaDEUUID,in_Geometry,in_GNISFeatureNameCV,in_WaterQualityIndicatorCV,in_WaterSourceName,in_WaterSourceNativeID,in_WaterSourceTypeCV,in_RegulatoryOverlayUUIDs,in_WaterSourceUUID,in_CoordinateAccuracy,in_CoordinateMethodCV,in_County,in_EPSGCodeCV,in_GNISCodeCV,in_HUC12,in_HUC8,in_Latitude,in_Longitude,in_NHDNetworkStatusCV,in_NHDProductCV,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SitePoint,in_SiteTypeCV,in_StateCV,in_USGSSiteID,in_MethodUUID,in_OrganizationUUID,in_SiteUUID,in_VariableSpecificUUID,in_AllocationApplicationDate,in_AllocationAssociatedConsumptiveUseSiteIDs,in_AllocationAssociatedWithdrawalSiteIDs,in_AllocationBasisCV,in_AllocationChangeApplicationIndicator,in_AllocationCommunityWaterSupplySystem,in_AllocationCropDutyAmount,in_AllocationExpirationDateID,in_AllocationFlow_CFS,in_AllocationLegalStatusCV,in_AllocationNativeID,in_AllocationOwner,in_AllocationPriorityDate,in_AllocationTimeframeEnd,in_AllocationTimeframeStart,in_AllocationTypeCV,in_AllocationVolume_AF,in_BeneficialUseCategory,in_CommunityWaterSupplySystem,in_CropTypeCV,in_CustomerTypeCV,in_DataPublicationDate,in_DataPublicationDOI,in_ExemptOfVolumeFlowPriority,in_GeneratedPowerCapacityMW,in_IrrigatedAcreage,in_IrrigationMethodCV,in_LegacyAllocationIDs,in_OwnerClassificationCV,in_PopulationServed,in_PowerType,in_PrimaryBeneficialUseCategory,in_SDWISIdentifierCV,in_WaterAllocationNativeURL
0,caD0,,,,WaDE_Unspecified,WaDECA_WS1,WaDE_Unspecified,,,,WaDE_Unspecified,,4326,,,,0.00000,0.00000,,,POD,WaDE_Unspecified,WaDE_Unspecified,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Closed,,,,,,Not Determined,0.00000,WaDE_Unspecified,,,,,,0,,0.00000,,,,0,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
1,caD1,,,,LAKE DOMINGO,WaDECA_WS2,Surface,,,,GIS_NE,San Diego,4326,,181002000000.00000,18100202.00000,32.61440,-116.27260,-116.27260,,POD,LAKE DOMINGO,CA60498,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,4/18/2013,,,,,,,,0.15000,Cancelled,T032025,"569 EAST COUNTY BOULEVARD, LLC",,31-Dec,1-Jan,Temporary Permit,0.00000,Dust Control,,,,,,0,,0.00000,,,,0,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
2,caD2,,,,ICY SPRING NO 2,WaDECA_WS3,WaDE_Unspecified,,,,DD_NE,Los Angeles,4326,,180902000000.00000,18090206.00000,34.57797,-118.28921,-118.28921,,POD,WaDE_Unspecified,CA34881,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Licensed,A000016,LILAC HILLS ESTATES LP,,31-Dec,1-Jan,Appropriative,0.00000,Domestic,,,,,,0,,0.00000,,,,0,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
3,caD3,,,,SACRAMENTO RIVER,WaDECA_WS4,WaDE_Unspecified,,,,DD_NE,Glenn,4326,,180202000000.00000,18020157.00000,39.79451,-122.05447,-122.05447,,POD,WaDE_Unspecified,CA28036,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Licensed,A000018,GLENN-COLUSA IRRIGATION DISTRICT,,1-Nov,1-Mar,Appropriative,0.00000,Irrigation,,,,,,0,,6047.70000,,,,0,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
4,caD4,,,,SAN JOAQUIN RIVER,WaDECA_WS5,WaDE_Unspecified,,,,DD_NE,Madera,4326,,180400000000.00000,18040001.00000,37.00260,-119.70722,-119.70722,,POD,WaDE_Unspecified,CA23233,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Licensed,A000023,US BUREAU OF RECLAMATION,,1-Jul,1-Apr,Appropriative,0.00000,Domestic,,,,,,0,,0.00000,,,,0,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65238,caD65238,,,,WaDE_Unspecified,WaDECA_WS1,WaDE_Unspecified,,,,WaDE_Unspecified,,4326,,,,0.00000,0.00000,,,POD,WaDE_Unspecified,WaDE_Unspecified,,WaDE_Unspecified,CA,,CAwr_M1,CAwr_O1,,CAwr_V1,,,,,,,,,0.00000,Active,UN008253,,,,,Not Determined,0.00000,WaDE_Unspecified,,,,,,0,,0.00000,,,,0,,,,https://ciwqs.waterboards.ca.gov/ciwqs/ewrims/...
65239,caD65239,,,,WaDE_Unspecified,WaDECA_WS1,WaDE_Unspecified,,,,

In [27]:
#Exporting to Finished File
outdf.to_csv('Pwr_CAMain.csv', index=False)  # The output